In [1]:
import ipywidgets as widgets 
from ipywidgets import AppLayout, Button, Layout
from ipywidgets import Layout, Button, Box, FloatText, Textarea, Dropdown, Label, IntSlider, DatePicker, Output, VBox, HBox, Text, SelectMultiple, Combobox, Accordion
from ipywidgets.embed import embed_data
import ipywidgets as widgets
from IPython.display import clear_output, HTML
import numpy as np
import geopandas as gpd
from webscraping_campgroundInfo import get_availability
import pandas as pd
import calplot
import numpy as np; np.random.seed(sum(map(ord, 'calplot')))
from collections import Counter
import datetime
import itertools

In [2]:
from interactive_mapping import interactive_facilities_map, interactive_campsite_map

campsite_nm = '../data/complete_campsites_CA.geojson'
facilities_nm = '../data/complete_facility_CA.geojson'

# [uncomment below] create facilities map and save as html 
# f = interactive_facilities_map(facilities_nm)

In [3]:
historical_searches = []

In [4]:
import threading
from IPython.display import display
import ipywidgets as widgets
import time
import matplotlib.pyplot as plt
%matplotlib inline

fac_map = open("../docs/facility_map.html", "rb")
image = HTML('<iframe src=../docs/facility_map.html width=650 height=600></iframe>')
facility_mapWidget = Output(layout=Layout(height = '40%', width='auto%'))
campsite_mapWidget = Output(layout=Layout(height = '50%', width='50%'))

start_dt = DatePicker(layout=Layout(flex='1 1 0%', width='auto'), disabled=False)
end_dt = DatePicker(layout=Layout(flex='1 1 0%', width='auto'), disabled=False)

user_facility_input = Combobox(
    # options = np.unique(historical_searches).tolist(), 
    placeholder='enter facility ID',
    description='Facility ID',
    ensure_option=False,
    disabled=False, style=dict(description_width='initial')
)

clear_button = widgets.Button(
    description='Clear outputs',
    disabled=False,
    button_style='info',layout = Layout(width='200px', height='auto'), 
    icon='eraser'
)

user_campgrounds_input = SelectMultiple(
            rows=5,
            description='CampsiteID',
            disabled=False
        )

Request_button = widgets.Button(
    description='Show Availability',
    disabled=False,
    button_style='success',
    icon='table', layout = Layout(width='200px', height='auto'),
    style=dict(description_width='initial'))

facility_button = widgets.Button(
    description='Search campsites',
    disabled=False,
    button_style='success',
    icon='search', layout = Layout(width='200px', height='auto'),
    style=dict(description_width='initial'))

directions = widgets.HTML(value='<details open> <summary>Instructions</summary> <p style="border:2px solid DodgerBlue;"> 1) Use the map on the right-hand side to identify the <i> Facility ID</i>. <br> 2) Input the <i>Facility ID</i> below and press <b>[ENTER]</b> key to display campgrounds. <br> 3) Once campsites of interests are chosen, input preferred start/ends dates, as well as the campsite name into fields to complete request.</p> </details>', layout=dict(width='auto', height='auto'))


def get_campsiteIDS(facilityid, campsite_nm = campsite_nm):
    campsite_df = gpd.read_file(campsite_nm)
    
    
    df = campsite_df[(campsite_df['FacilityID'] == facilityid) & \
            (campsite_df['Reservable'] == True) &\
                (campsite_df['TypeOfUse']== type_use.value) &\
                    (campsite_df['Max Num of People'] >= num_people.value) &\
                            (campsite_df['Max Num of Vehicles'] >= num_vehicles.value) &\
                                    (campsite_df["EquipmentsAllowed"].str.contains(equipment.value, na=False))]
    
    if pets_allowed.value == True: 
        df = df[df['Pets Allowed'].isin(['Yes' , 'Pets Allowed', 'Domestic,Horse', 'Horse'])]
    
    elif pets_allowed.value == False: 
        df = df[df['Pets Allowed'].isin(['Yes' , 'Pets Allowed', 'Domestic,Horse', 'Horse', 'No', None])]


    campsite_lst = df['CampsiteName'].tolist()
    return campsite_lst

def get_facility_nm(facilityid, facility_nm = facilities_nm): 
    facility_df = gpd.read_file(facility_nm)
    df = facility_df[facility_df['FacilityID'] == facilityid]
    FacilityName = df.iloc[0]['FacilityName']
    return FacilityName

with facility_mapWidget:
    display(image)

def get_calender(start_dt, end_dt, df, facility_name):
    a = df['Available']
    all_avail = list(itertools.chain.from_iterable(a))

    idx = pd.date_range(start_dt, end_dt)
    count_df = pd.DataFrame.from_dict({'Date':list(Counter(all_avail).keys()), 'Count':list(Counter(all_avail).values())})
    count_df['Date'] = pd.to_datetime(count_df['Date'], infer_datetime_format=True)
    count_df = count_df.set_index('Date')    
    count_df = count_df.reindex(idx, fill_value=-1)

    events = pd.Series(count_df['Count']) 
    calplot.calplot(events, edgecolor=None, suptitle=f'Campsite Availability for {facility_name}', textfiller='-', cmap='RdYlGn', colorbar=False)
    return(calplot)

output_success = widgets.Output(layout=dict(height= '100%', width='100%'))

progress_request = widgets.FloatProgress(value=0.0, min=0.0, max=1.0, 
    description='Loading',
    bar_style='info',
    style={'bar_color': '#a98f9f'})
Complete_msg_request = widgets.HTML()

def work_2(progress_request):
    Complete_msg_request.value = ""
    total = 50
    for i in range(total):
        time.sleep(0.2)
        progress_request.value = float(i+1)/total
    Complete_msg_request.value = f"<h4 style='color:MediumSeaGreen;'> Proceed to next tab! </h4>"

def get_facilityInfo(facilityid, facility_nm = facilities_nm):
    campsite_df = gpd.read_file(facilities_nm)
    df = campsite_df[(campsite_df['FacilityID'] == facilityid)]
    html = df['FacilityDescription'].iloc[0]
    return html

campground_info = widgets.Output(layout=dict(height= '50%', width='50%'))

def on_request_button_clicked(Request_button):
    thread = threading.Thread(target=work_2, args=(progress_request,))
    clear_output()
    display(progress_request)
    thread.start()
    avail_df = get_availability(str(user_facility_input.value), list(user_campgrounds_input.value), start_dt.value, end_dt.value, campsite_nm)
    with campground_info: 
        display(HTML(f"""<details> <summary>Campground Info: {get_facility_nm(str(user_facility_input.value)).title()} </summary> <h1>{get_facility_nm(str(user_facility_input.value)).title()}</h1> <br> {get_facilityInfo(str(user_facility_input.value))}"""))
    with output_success: 
        if len(avail_df['Available']) > 1: 
            plt.show(get_calender(start_dt.value, end_dt.value, avail_df, get_facility_nm(str(user_facility_input.value))))
        else: 
            display(HTML(f"""<details> <summary>Status </summary> <p style="border:2px solid Tomato;"> No reservable dates for <b>{get_facility_nm(str(user_facility_input.value)).title()} </b> based on your search criteria. Try a wider time range, or a different campground.</p> </details>""")) 
        
    
progress_search = widgets.FloatProgress(value=0.0, min=0.0, max=1.0, 
    description='Loading',
    bar_style='info',
    style={'bar_color': '#94b79f'})
Complete_msg = widgets.HTML()


def work(progress_search):
    Complete_msg.value = ""
    total = 25
    for i in range(total):
        time.sleep(0.1)
        progress_search.value = float(i+1)/total
    Complete_msg.value = f"<h4 style='color:MediumSeaGreen;'> Proceed to next tab! </h4>"

header = widgets.HTML()


def callback(wdgt):
    header.value = f'<h2>{get_facility_nm(str(user_facility_input.value))}</h2>'
    thread = threading.Thread(target=work, args=(progress_search,))
    display(progress_search)
    thread.start()
    with user_campgrounds_input.hold_trait_notifications(): 
        lst = get_campsiteIDS(str(user_facility_input.value))
        user_campgrounds_input.options = lst
    with campsite_mapWidget:
        clear_output()
        display(interactive_campsite_map(lst, campsite_nm, str(user_facility_input.value)))
        historical_searches.append(str(user_facility_input.value))
    with user_facility_input.hold_trait_notifications(): 
        user_facility_input.options = np.unique(historical_searches).tolist()

def on_clear_button_clicked(clear_button):
    with output_success: 
        clear_output()
    with campground_info: 
        clear_output()

facility_button.on_click(callback)
clear_button.on_click(on_clear_button_clicked)
Request_button.on_click(on_request_button_clicked)
user_facility_input.on_submit(callback)


/Users/erica/Library/Python/3.8/lib/python/site-packages/IPython/core/display.py:717: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


In [5]:
num_people = widgets.IntSlider(
    value=0, min=0, max=100, step=1,
    description='Number of People:',
    disabled=False,
    orientation='horizontal',
    
    readout=True, style=dict(description_width='initial')
)

num_vehicles = widgets.IntSlider(
    value=0, min=0, max=10, step=1,
    description='Number of Vehicles:',
    disabled=False,
    orientation='horizontal',
    readout=True, style=dict(description_width='initial')
)

equipment = widgets.ToggleButtons(
    value='Tent', options=['RV', 'Tent', 'Trailer'],
    description= 'Equipment',
    disabled=False
)

type_use = widgets.RadioButtons(
    options=['Overnight', 'Day-Use Only'],
   value='Overnight',
    description='Use Type:',
    disabled=False
)

pets_allowed = widgets.Checkbox(
    value=False,
    description='Pets Allowed',
    disabled=False,
    indent=False
)

In [6]:
tab1 = HBox(children=[VBox(children=[directions, user_facility_input, type_use, pets_allowed, num_people, num_vehicles, equipment, facility_button, HBox(children=[progress_search, Complete_msg]) ]),
                      facility_mapWidget
                      ])
tab2 = VBox(children=[header, HBox(children=[VBox(children=[Label(value='Input criteria to set campsite availability alerts below.', layout=dict(width='auto', height='auto')), 
        HBox(children=[Label(value='Start Date:', layout=dict(height='auto')), start_dt, 
                    Label(value='End Date:', layout=dict(height='auto')), end_dt ]), user_campgrounds_input, HBox(children=[Request_button]), HBox(children=[progress_request, Complete_msg_request]) ]), campsite_mapWidget ])])

tab3 = VBox(children=[clear_button, HBox(children=[output_success, campground_info])])

In [7]:
tab_nest = widgets.Tab()
tab_nest.children = [tab1, tab2, tab3]
tab_nest.set_title(0, 'Select Facility')
tab_nest.set_title(1, 'Search Campsites')
tab_nest.set_title(2, 'Availability')
tab_nest